In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# All project files live here in your Google Drive
PROJECT_DIR = "/content/drive/MyDrive/phys2600_cosmic_project"
%cd $PROJECT_DIR

%matplotlib inline
from cosmic import *
import matplotlib.pyplot as plt

## 1. `lorentz_beta_gamma`

Verify that `lorentz_beta_gamma` computes the correct values of $\beta$ and $\gamma$.

__Test input:__ `(beta_x, beta_y) = (0.3, 0.4)`

__Expected output:__

$$
\beta = 0.5 \\
\gamma = 1/\sqrt{1-\beta^2} = 2/\sqrt{3} \approx 1.1547
$$

In [ ]:
test_beta_xy = (0.30, 0.40)

test_beta, test_gamma = lorentz_beta_gamma(*test_beta_xy)
print(test_beta, test_gamma)

assert np.abs(test_beta - 0.5) <= 1e-6
assert np.abs(test_gamma - 1.15470) <= 1e-6

## 2. `boost_momentum`

Using the test beta/gamma from above, boost a known momentum to another frame of reference.

__Test input:__ `(p_x, p_y) = (40, -40) MeV/c`, for a pion (mass `m = 139.6 MeV/c**2`.)  Same `(beta_x, beta_y) = (0.3, 0.4)` from above.

__Expected output:__  (calculated by hand)

Energy: $E = \sqrt{m^2 + p^2} \approx 150.63$ MeV

$(p_x', p_y') = (-12.92, -110.56)$

In [ ]:
test_p_xy = (40., -40.)
test_p = np.sqrt(test_p_xy[0]**2 + test_p_xy[1]**2)
test_m = 139.6 

test_E = np.sqrt(test_p**2 + test_m**2 )
print(test_E)

px_boost, py_boost = boost_momentum(test_m, test_p_xy[0], test_p_xy[1], test_beta_xy[0], test_beta_xy[1])
print([px_boost, py_boost] )

assert np.abs(test_E - 150.63)/150.63 <= 1e-4
assert np.abs(px_boost + 12.92)/12.92 <= 1e-4
assert np.abs(py_boost + 110.56)/110.56 <= 1e-4


## 3. `check_for_interaction`

Several test cases for interactions with very high/low probability, so the outcome should always be True/False.

In [ ]:
h = 15 * 1000 / 3e8 # km --> light-seconds

test_p1 = velocity_to_momentum(particle_mass('electron'), 0.3, 0.4)
test_part_1 = ('electron', test_p1[0], test_p1[1], [], [])
print("Electron interacts:  (must be False)")
print(check_for_interaction(test_part_1, 1e6, h))

test_p2 = velocity_to_momentum(particle_mass('muon'), 0.3, 0.4)
test_part_2 = ('muon', test_p2[0], test_p2[1], [], [])
print("Muon interacts in 1e-10 seconds: (False)")
print(check_for_interaction(test_part_2, 1e-10, h))

print("Muon interacts in 100 seconds: (True)")
print(check_for_interaction(test_part_2, 10, h))

test_p3 = velocity_to_momentum(particle_mass('proton'), 0.7, 0.4)
test_part_3 = ('proton', test_p3[0], test_p3[1], [], [])
print("Proton interacts in 1 second: (True)")
print(check_for_interaction(test_part_3, 1 , h))

test_p4 = velocity_to_momentum(particle_mass('proton'), 0.3, 0.2)
test_part_4 = ('proton', test_p4[0], test_p4[1], [], [])
print("Slow proton doesn't interact (below threshold): (False)")
print(check_for_interaction(test_part_4, 1, h ))

## 4. `proton_downscatter`

Verify that proton downscattering returns decay-product tuples, and that final momentum is in the expected direction in the proton's rest frame.

In [ ]:
# Note: this is above threshold energy, you will get NaN sometimes if it isn't!
part1, part2, part3 = proton_downscatter(0.5, 0.7)  

# Should be a proton and two pions as decay product tuples
print(part1,'\n', part2, '\n', part3)

# Check final momentum: net momentum should be in the opposite direction of the boost vector,
# which determines the direction of the incident N-14 nucleus in the proton rest frame

px_net = part1[2] + part2[2] + part3[2]
py_net = part1[3] + part2[3] + part3[3]
p_net_angle = np.arctan2(py_net, px_net)

print("Angle of net momentum vector: ", p_net_angle)
print("Expected angle: ", np.arctan2(0.7, 0.5) - np.pi)

assert np.abs(p_net_angle - np.arctan2(0.7, 0.5) + np.pi) <= 1e-6
